В этом степе вам предстоит дополнить свою рекомендательную систему с помощью новых знаний, которые вы приобрели на блоке про глубинное обучение.
Возможные направления работы
В качестве возможных направлений для улучшения своей рекомендательной системы предлагаем вам рассмотреть:
1.	Дополнительные признаки от нейронных сетей, например, векторизовать тексты постов с помощью разных методов, рассмотренных в блоке, сделать какие-то новые признаки на основе этих векторов.
2.	Применить нейронные сети в качестве модели рекомендаций.
3.	Обучить какие-то нестандартные подходы, например, попробовать построить векторы пользователей.

В данном конкретно случае для текстов постов построены эмбеддинги с помощью модели Bert. Код с построением эмбеддингов находится в ноутбуке Get_embeddings.

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [3]:
# Загружаем таблицу с фичами пользователей
user_features = pd.read_csv('user_features.csv')

In [6]:
user_features.head(3)

,Unnamed: 0,user_id,post_id,target,gender,age,country,city,exp_group,os,source,user_rate,text,topic,post_rate,timestamp
0,37258,78733,1693,0,1,45,Russia,Perm,0,Android,ads,0.140271,Merritt close to indoor 400m mark\n\nTeenager ...,sport,0.181818,2021-10-01 06:12:22
1,70923,155614,1693,0,1,32,Russia,Kursk,2,Android,organic,0.193548,Merritt close to indoor 400m mark\n\nTeenager ...,sport,0.181818,2021-10-01 06:12:22
2,913,1427,1693,0,1,25,Russia,Yartsevo,3,iOS,ads,0.120996,Merritt close to indoor 400m mark\n\nTeenager ...,sport,0.181818,2021-10-01 06:12:22


In [5]:
# Загружаем таблицу с постами и посчитанными эмбедиингами для текстов

new_post_df = pd.read_csv('new_post_df.csv')

In [7]:
new_post_df.head(3)

,Unnamed: 0,post_id,text,topic,0,1,2,3,4,5,...,758,759,760,761,762,763,764,765,766,767
0,0,1,UK economy facing major risks\n\nThe UK manufa...,business,0.140364,-0.140696,-0.575681,-0.118175,-0.315324,-0.114377,...,0.051768,0.614536,-0.613106,0.131061,0.202857,0.175931,-0.167768,-0.137881,0.042959,0.142283
1,1,2,Aids and climate top Davos agenda\n\nClimate c...,business,0.157530,-0.097739,-0.230651,-0.364431,-0.242782,0.310065,...,0.586033,0.652154,-0.112308,-0.085171,-0.051814,0.240047,0.200299,-0.300891,0.190542,0.019754
2,2,3,Asian quake hits European shares\n\nShares in ...,business,0.314569,-0.115163,-0.181323,-0.274698,-0.357378,0.285229,...,0.416196,0.641697,-0.326961,-0.042803,-0.073850,0.212023,-0.090192,-0.354049,-0.204324,-0.027026


In [9]:
# удаляем лишние колонки, чтобы не было дубликатов

new_post_df = new_post_df.drop(['Unnamed: 0', 'text', 'topic'], axis=1)

In [10]:
# объединяем таблицы
user_features = user_features.merge(new_post_df, on='post_id', how='left')
user_features.head()

,Unnamed: 0,user_id,post_id,target,gender,age,country,city,exp_group,os,...,758,759,760,761,762,763,764,765,766,767
0,37258,78733,1693,0,1,45,Russia,Perm,0,Android,...,0.080003,0.665498,-0.2153,-0.114099,0.015371,0.21538,0.070674,-0.267132,0.012114,0.167146
1,70923,155614,1693,0,1,32,Russia,Kursk,2,Android,...,0.080003,0.665498,-0.2153,-0.114099,0.015371,0.21538,0.070674,-0.267132,0.012114,0.167146
2,913,1427,1693,0,1,25,Russia,Yartsevo,3,iOS,...,0.080003,0.665498,-0.2153,-0.114099,0.015371,0.21538,0.070674,-0.267132,0.012114,0.167146
3,61074,128641,1693,0,0,20,Russia,Ulan-Ude,3,iOS,...,0.080003,0.665498,-0.2153,-0.114099,0.015371,0.21538,0.070674,-0.267132,0.012114,0.167146
4,74681,1431,1693,1,1,25,Russia,Mirnyy,1,iOS,...,0.080003,0.665498,-0.2153,-0.114099,0.015371,0.21538,0.070674,-0.267132,0.012114,0.167146


In [12]:
user_features = user_features.drop('Unnamed: 0', axis=1)
user_features.to_csv('user_features.csv')

In [17]:
### Разделим на трейн-тест
user_features = user_features.sort_values(by='timestamp')

train = user_features.iloc[:-20000].copy()
test = user_features.iloc[-20000:].copy()

In [18]:
train_new = train.drop(['user_id', 'post_id',  
                         'exp_group', 'text', 'timestamp'], axis=1) 

test_new = test.drop(['user_id','post_id',  
                        'exp_group',  'text', 'timestamp'], axis=1) 

# удаляем лишние колонки. exp_group будет использована в АВ тестировании.

In [19]:
X_train = train_new.drop('target', axis=1)
X_test = test_new.drop('target', axis=1)

y_train = train['target']
y_test = test['target']

In [20]:
cat_features=['gender', 'country', 'city', 'os', 'source', 'topic']

In [ ]:
# Строим и обучаем модель

from catboost import CatBoostClassifier

new_catboost = CatBoostClassifier()


new_catboost.fit(X_train,
             y_train,
             cat_features=cat_features,
             )

In [22]:
test_new = user_features.iloc[-20000:].copy()

X_test['prob'] = new_catboost.predict_proba(X_test)[:, 1]
X_test['pred'] = new_catboost.predict(X_test)
X_test['target'] = y_test
X_test['user_id'] = test_new['user_id']
X_test['post_id'] = test_new['post_id']

In [23]:
hitrate5 = []

for user in X_test['user_id'].unique():
    part = X_test[X_test['user_id']==user]
    if sum(part['pred']==1)==0:
        continue # пропускаем пользователя, если для него нет рекомендаций
    elif sum(part[part['pred']==1].sort_values(by='prob', ascending=False)[:5]['target']==1)>0:
        hitrate5.append(1)
    else:
        hitrate5.append(0)
        
print(f"Среднее HitRate@5 по пользователям из теста: {np.mean(hitrate5)}")

Среднее HitRate@5 по пользователям из теста: 1.0


Таким образом, добавление эмбеддингов постов к датасету позволило значительно улучшить качество модели.

In [24]:
new_catboost.save_model('new_catboost_model',
                           format="cbm")